In [46]:
from langgraph.graph import StateGraph,START,END
from langgraph.types import Command,interrupt
from typing import  TypedDict
from langgraph.checkpoint.memory import MemorySaver

In [47]:
memory = MemorySaver()

In [48]:
class State(TypedDict):
    value: str

In [58]:
def node_a(state: State):
    print("Node A")
    return Command(
        goto="node_b",
        update={
            "value":state["value"] + "a"
        }
    )

In [59]:
def node_b(state: State):
    print("node B")

    human_response = interrupt("Do you want to got to C or D? Type C/D")
    print("Human Review Values:", human_response)

    if(human_response=="C"):
        return Command(
        goto="node_c",
        update={
            "value":state["value"] + "b"
        }
    )

    elif(human_response=="D"):
        return Command(
        goto="node_d",
        update={
            "value":state["value"] + "b"
        }
    )


In [60]:
def node_c(state: State):
    print("Node C")
    return Command(
        goto=END,
        update={
            "value":state["value"] + "c"
        }
    )

def node_d(state: State):
    print("Node D")
    return Command(
        goto=END,
        update={
            "value":state["value"] + "d"
        }
    )

In [61]:
graph = StateGraph(State)

graph.add_node("node_a",node_a)
graph.set_entry_point("node_a")
graph.add_node("node_b",node_b)
graph.add_node("node_c",node_c)
graph.add_node("node_d",node_d)

app = graph.compile(checkpointer=memory)

In [62]:
config = {"configurable":{
    "thread_id":"1"
}}

initialState = {
    "value":""
}

In [63]:
first_result = app.invoke(initialState,config=config,stream_mode="updates")
first_result

Node A
node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to got to C or D? Type C/D', resumable=True, ns=['node_b:42a25060-8109-2f3e-3cb3-44e93743a42d']),)}]

In [64]:
print(app.get_state(config=config).next)

('node_b',)


In [65]:
second_result = app.invoke(Command(resume="D"),config=config,stream_mode='updates')

node B
Human Review Values: D
Node D


In [66]:
second_result

[{'node_b': {'value': 'ab'}}, {'node_d': {'value': 'abd'}}]